## Задание 1
Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:
- оценка 2 и меньше - низкий рейтинг
- оценка 4 и меньше - средний рейтинг
- оценка 4.5 и 5 - высокий рейтинг

Результат классификации запишите в столбец class

In [1]:
import pandas as pd

In [28]:
ratings = pd.read_csv('ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [25]:
# группируем данные ratings по movieId и рассчитываем средний рейтинг по всем пользователям для каждого фильма
movie_rating = ratings.groupby('movieId').mean()
movie_rating.head()

,userId,rating,timestamp
movieId,,,
1,338.558704,3.872470,1.103116e+09
2,318.906542,3.401869,1.069321e+09
3,374.423729,3.161017,9.662429e+08
4,355.538462,2.384615,9.277797e+08
5,320.785714,3.267857,9.967201e+08


In [26]:
def rating_level(x):
    """
    Функция классифицирует фильмы в терминах высокий/средний/низкий рейтинг
    """
    if x <= 2:
        return 'low rating'
    elif x <= 4:
        return 'average rating'
    elif x <= 5:
        return 'high rating'

In [30]:
# применяем функцию к группировке movie_rating, добавляя новый столбец class
movie_rating['class'] = movie_rating['rating'].apply(rating_level)
movie_rating['class'].head(10)

movieId
1     average rating
2     average rating
3     average rating
4     average rating
5     average rating
6     average rating
7     average rating
8     average rating
9     average rating
10    average rating
Name: class, dtype: object

### Задание 2
Используем файл keywords.csv.

Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону. Т. е. если поисковый запрос содержит название города региона, то в столбце ‘region’ пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим ‘undefined’.

Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:

geo_data = {

1
2
3
4
5
'Центр': ['москва', 'тула', 'ярославль'],

'Северо-Запад': ['петербург', 'псков', 'мурманск'],

'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}

Результат классификации запишите в отдельный столбец region.

In [4]:
keywords = pd.read_csv('keywords.csv')
keywords

,keyword,shows
0,вк,64292779
1,одноклассники,63810309
2,порно,41747114
3,ютуб,39995567
4,вконтакте,21014195
...,...,...
99995,эльдорадо старый оскол,3705
99996,frigate для firefox,3630
99997,укрсиб,3630
99998,погода в ялте на неделю,3688


In [3]:
geo_data = {'Центр': ['москва', 'тула', 'ярославль'],
            'Северо-Запад': ['петербург', 'псков', 'мурманск'],
            'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']}

In [5]:
def region_search(row):
       
    for region, cities in geo_data.items():
        for city in cities:
            if city in row:
                return region
        
    return 'undefined'

In [12]:
keywords['region'] = keywords['keyword'].apply(region_search)
keywords.head(10)

,keyword,shows,region
0,вк,64292779,undefined
1,одноклассники,63810309,undefined
2,порно,41747114,undefined
3,ютуб,39995567,undefined
4,вконтакте,21014195,undefined
5,одноклассники моя страница,19976138,undefined
6,майл,15144673,undefined
7,авито,14786593,undefined
8,переводчик,14453469,undefined
9,яндекс,13344734,undefined


In [10]:
# проверим, как сработала функция, в частности определился ли где-то регион. 
# для этого отфильтруем по колонке region значения, отличные от undefined

keywords[~keywords['region'].isin(['undefined'])]

,keyword,shows,region
127,авито москва,979292,Центр
370,авито ру санкт петербург,425134,Северо-Запад
564,погода в санкт петербурге,292489,Северо-Запад
849,авито ярославль,209581,Центр
1063,фарпост владивосток,176951,Дальний Восток
...,...,...,...
99590,авито ярославль автомобили с пробегом,3673,Центр
99634,северпост новости мурманской области,3606,Северо-Запад
99808,полармед мурманск запись на прием,3719,Северо-Запад
99890,яндекс метро москва,4548,Центр


## Задание 3 (бонусное)

Есть мнение, что “раньше снимали настоящее кино, не то что сейчас”. Ваша задача проверить это утверждение, используя файлы с рейтингами фильмов из прошлого домашнего занятия (файл movies.csv и ratings.csv из базы https://grouplens.org/datasets/movielens). Т. е. проверить верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.

При этом мы не будем затрагивать субьективные факторы выставления этих рейтингов, а пройдемся по следующему алгоритму:

1) В переменную years запишите список из всех годов с 1950 по 2010.

2) Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте следующий алгоритм:

  - для каждой строки пройдите по всем годам списка years
  - если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска
  - если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год

3) Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец ‘year’

4) Посчитайте средний рейтинг всех фильмов для каждого значения столбца ‘year’ и отсортируйте результат по убыванию рейтинга

In [20]:
movies = pd.read_csv('movies.csv')
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9120,162672,Mohenjo Daro (2016),Adventure|Drama|Romance
9121,163056,Shin Godzilla (2016),Action|Adventure|Fantasy|Sci-Fi
9122,163949,The Beatles: Eight Days a Week - The Touring Y...,Documentary
9123,164977,The Gay Desperado (1936),Comedy


In [17]:
# промежуток времени, который нас интересует для изучения рейтингов 
# с 1950 по 2021 (текущий год, чтобы по максимуму захватить новые фильмы)
years = range(1950, 2022)

In [23]:
def production_year(row):
    """
    Функция production_year выставляет год выпуска для каждой строки из названия фильма. 
    Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используется следующий алгоритм:
     - для каждой строки пройдите по всем годам списка years
     - если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска
     - если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год
    """
    for year in years:
        if str(year) in row:
            return year
    
    return '1900'

In [25]:
# добавляем столбец с годом в таблицу movies
movies['year'] = movies['title'].apply(production_year)
movies

,movieId,title,genres,year
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji (1995),Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men (1995),Comedy|Romance,1995
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II (1995),Comedy,1995
...,...,...,...,...
9120,162672,Mohenjo Daro (2016),Adventure|Drama|Romance,2016
9121,163056,Shin Godzilla (2016),Action|Adventure|Fantasy|Sci-Fi,2016
9122,163949,The Beatles: Eight Days a Week - The Touring Y...,Documentary,2016
9123,164977,The Gay Desperado (1936),Comedy,1900


In [34]:
# присоединяем ratings к movies
movies_ratings = pd.merge(movies, ratings, on='movieId')
movies_ratings

,movieId,title,genres,year,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995,7,3.0,851866703
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995,9,4.0,938629179
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995,13,5.0,1331380058
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995,15,2.0,997938310
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995,19,3.0,855190091
...,...,...,...,...,...,...,...
99999,161944,The Last Brickmaker in America (2001),Drama,2001,287,5.0,1470167824
100000,162376,Stranger Things,Drama,1900,73,4.5,1474255532
100001,162542,Rustom (2016),Romance|Thriller,2016,611,5.0,1471520667
100002,162672,Mohenjo Daro (2016),Adventure|Drama|Romance,2016,611,3.0,1471523986


In [43]:
# группируем по году выхода, ищем средний рейтинг для каждого года, сортируем по убыванию параметра 'rating'
movies_ratings.groupby('year').mean().sort_values(by='rating', ascending=False)['rating']

year
1957    4.014241
1972    4.011136
1952    4.000000
1954    3.994220
1951    3.983539
          ...   
2011    3.401188
2015    3.371028
2013    3.357399
2000    3.356104
2016    3.217742
Name: rating, Length: 68, dtype: float64